In [1]:
import tensorflow as tf
import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,Input
from keras import backend as K
from keras.callbacks import TensorBoard

Using TensorFlow backend.


### Mirar las GPU's disponibles

In [8]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [2]:
batch_size = 100
num_classes = 10
epochs = 10

In [3]:
filas,columnas = 28, 28

(xt,yt), (xtest,ytest) = mnist.load_data()

xt = xt.reshape(xt.shape[0], filas, columnas,1)
xtest = xtest.reshape(xtest.shape[0], filas, columnas, 1)

xt = xt.astype('float32')
xtest = xtest.astype('float32')

xt = xt / 255
xtest = xtest / 255

yt = keras.utils.to_categorical(yt, num_classes)
ytest = keras.utils.to_categorical(ytest, num_classes)

## Creación de las Capas

In [4]:
Entradas = Input(shape=(28, 28, 1))

# Capa convolucional a las entradas, 64 filtros con tamaño de 3x3
x = Conv2D(64, kernel_size=(3,3), activation='relu')(Entradas)

# Capa convolucional a la salida de la primera convolución, 128 filtros con tamaño de 3x3
x = Conv2D(128,kernel_size=(3,3),activation='relu')(x)

# Reducir el tamaño de la salida y obtener las características más importantes
x = MaxPooling2D(pool_size=(2,2))(x)  

# Reducir a una dimensión
x = Flatten()(x)

# Capas Clásicas
x = Dense(68,activation='relu')(x)
#x=Dropout(0.25)(x) # Opcional
x = Dense(20,activation='relu')(x)
#x=Dropout(0.25)(x)
x = Dense(num_classes,activation='softmax')(x)

Instructions for updating:
Colocations handled automatically by placer.


## Creación del Modelo

In [5]:
modelo = Model(inputs=Entradas, outputs=x)
modelo.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 68)                1253444   
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1380      
__________

## Compilación y entrenamiento del modelo

In [6]:
modelo.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['categorical_accuracy'])

# Entrenar preferiblemente con GPU, sino tardará bastante (+- 3 minutos por época)
modelo.fit(xt, yt, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(xtest,ytest))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 11s 179us/step - loss: 0.1699 - categorical_accuracy: 0.9491 - val_loss: 0.0602 - val_categorical_accuracy: 0.9826
Epoch 2/10
60000/60000 [==============================] - 7s 113us/step - loss: 0.0443 - categorical_accuracy: 0.9866 - val_loss: 0.0372 - val_categorical_accuracy: 0.9877
Epoch 3/10
60000/60000 [==============================] - 7s 113us/step - loss: 0.0288 - categorical_accuracy: 0.9909 - val_loss: 0.0508 - val_categorical_accuracy: 0.9843
Epoch 4/10
60000/60000 [==============================] - 7s 114us/step - loss: 0.0190 - categorical_accuracy: 0.9940 - val_loss: 0.0396 - val_categorical_accuracy: 0.9885
Epoch 5/10
60000/60000 [==============================] - 7s 115us/step - loss: 0.0146 - categorical_accuracy: 0.9953 - val_loss: 0.0375 - val_categorical_accuracy: 0.9893
Epoch 6/10
60000/60000 [=================

## Evaluación del modelo

In [7]:
puntuacion = modelo.evaluate(xtest, ytest, verbose=1)
puntuacion

10000/10000 [==============================] - 1s 69us/step


[0.0591607699227804, 0.9864]